## 1-5 fine_tuning 으로 정확도를 향상시키는 방법

- 학습된 VGG 모델을 사용해서 fine_tuning으로 개미와 벌을 이미지 분류하는 모델을 학습시키기

### 사전준비

In [1]:
# 패키지 Import 
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models
from tqdm import tqdm

c:\Users\jx778\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## Dataset과 DataLoader 작성

In [10]:
#import sys
# sys.path.append('C:\pythonStudy\Pytorch_Flipped_Learning\CH1_image_classification')
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 거미와 벌의 이미지 파일 경로 리스트 작성
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
    file_list = train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list = val_list, transform = ImageTransform(size, mean, std), phase='val'
)

batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


## 네트워크 모델 작성하기

In [11]:
# 학습된 VGG-16 모델 로드하기
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#vgg16 의 마지막 출력층 유닛을 개마, 벌 총 2개로 바꿔주기
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# trian모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다')

네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다


## 손실함수 정의

In [12]:
# 손실 함수의 설정
criterion = nn.CrossEntropyLoss()